# Note Book To Fix Country Boarder Problem

This notebook is to clip polygons that overlap country boundaries

By Cascade Tuholske and Ryan Avery 2019-04-29

### Dependencies

In [2]:
from rasterstats import zonal_stats
import rasterio
import geopandas as gpd
import operator
import numpy as np
import pandas as pd
import matplotlib 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from functools import reduce
from scipy import stats as ss

### Load Data

In [24]:
# Paths
data_raw = '../../data/raw/'
data_temp = '../../temp_data/'
data_interim = '../../interim/'
erl_data = '../../temp_data/ERL19/ERL_data/'
downloads = ''
erl_v2_data = '' 

In [25]:
GHS2000 = gpd.read_file(erl_v2_data+'GHS2000_20190429_all.shp', driver = 'ESRI Shapefile')
GHS2015 = gpd.read_file(erl_v2_data+'GHS2015_20190429_all.shp', driver = 'ESRI Shapefile')
WP2015 = gpd.read_file(erl_v2_data+'WP2015_20190429_all.shp', driver = 'ESRI Shapefile')
LS2015 = gpd.read_file(erl_v2_data+'WPE2016_20190429_all.shp', driver = 'ESRI Shapefile')
WPE2016 = gpd.read_file(erl_v2_data+'LS2015_20190429_all.shp', driver = 'ESRI Shapefile')

In [54]:
def find_dup(df, col1):
    df_out = gpd.GeoDataFrame()
    df.drop_duplicates([col1])
#     duplicated = df.duplicated([col1,col2], False)
    
#     df_out = df[duplicated]
    return df_out

In [55]:
# Test

num = [1,1,2,3,3,1]
state = ['mt', 'ca', 'mt', 'pa', 'pa', 'mt']
cool = ['yes', 'yes', 'yes', 'no', 'yes', 'yes']

test = pd.DataFrame()
test['FID'] = num
test['state'] = state
test['cool'] = cool

test

,FID,state,cool
0,1,mt,yes
1,1,ca,yes
2,2,mt,yes
3,3,pa,no
4,3,pa,yes
5,1,mt,yes


In [64]:
test1 = GHS2000.drop_duplicates(['FID','country'], False)
test1

,osm_id,FID,country,city,osm_type,lat,lon,PopTot,aez_class,geometry
0,89369215,5382,Algeria,Tamanrasset,town,22.785454,5.532446,2.833428e+04,Tropic - cool / arid,"POLYGON ((5.512930562910484 22.80475188764244,..."
2,253167052,195,Algeria,Thenia,town,36.724986,3.556935,2.574931e+04,Subtropic - warm / subhumid,"POLYGON ((3.610972972741117 36.7503307638946, ..."
3,253167208,150,Algeria,Zemmouri,town,36.786406,3.601221,1.952352e+04,Subtropic - warm / subhumid,"POLYGON ((3.555512812300016 36.81191901602259,..."
4,253291208,436,Algeria,Lakhdaria,town,36.563944,3.596907,5.083199e+04,Subtropic - warm / subhumid,"POLYGON ((3.548917172615572 36.58325611006445,..."
6,253292625,56,Algeria,Dellys,town,36.915798,3.913104,2.577718e+04,Subtropic - warm / subhumid,"POLYGON ((3.879862753266941 36.92634664178819,..."
7,258799889,4736,Algeria,El Menia,town,30.583668,2.883089,4.559333e+04,Subtropic - warm / arid,"POLYGON ((2.873234846555186 30.60876048327147,..."
8,262393185,5427,Algeria,In Guezzam,town,19.566724,5.771700,5.523000e+03,Tropic - warm / arid,"POLYGON ((5.769834985642884 19.56494482203358,..."
9,262963952,3194,Algeria,Benaceur,town,33.110521,6.442111,8.602000e+03,Subtropic - warm / arid,"POLYGON ((6.430187004128339 33.1131937646665, ..."
10,262969007,2034,Algeria,Ben Srour,town,35.040443,4.563379,1.998621e+04,Subtropic - warm / semiarid,"POLYGON ((4.545688066077435 35.05066625912546,..."
11,264023963,1547,Algeria,Chahbounia,town,35.541541,2.603774,1.142530e+04,Subtropic - warm / subhumid,"POLYGON ((2.590054993457434 35.54863558536618,..."


In [78]:
print(len(GHS2000))

GHS2000['dup'] = GHS2000['FID'].astype(str) + GHS2000['country']
GHS2000.head(50)


5854


,osm_id,FID,country,city,osm_type,lat,lon,PopTot,aez_class,geometry,dup
0,89369215,5382,Algeria,Tamanrasset,town,22.785454,5.532446,2.833428e+04,Tropic - cool / arid,"POLYGON ((5.512930562910484 22.80475188764244,...",5382Algeria
1,252600742,624,Algeria,Boumerdès,town,36.758882,3.470596,3.873888e+06,Subtropic - warm / subhumid,"POLYGON ((2.960095682991406 36.82071885667165,...",624Algeria
2,253167052,195,Algeria,Thenia,town,36.724986,3.556935,2.574931e+04,Subtropic - warm / subhumid,"POLYGON ((3.610972972741117 36.7503307638946, ...",195Algeria
3,253167208,150,Algeria,Zemmouri,town,36.786406,3.601221,1.952352e+04,Subtropic - warm / subhumid,"POLYGON ((3.555512812300016 36.81191901602259,...",150Algeria
4,253291208,436,Algeria,Lakhdaria,town,36.563944,3.596907,5.083199e+04,Subtropic - warm / subhumid,"POLYGON ((3.548917172615572 36.58325611006445,...",436Algeria
5,253292622,257,Algeria,Draâ Ben Khedda,town,36.733332,3.958769,3.368617e+05,Subtropic - warm / subhumid,"POLYGON ((3.842424792309737 36.82951907750195,...",257Algeria
6,253292625,56,Algeria,Dellys,town,36.915798,3.913104,2.577718e+04,Subtropic - warm / subhumid,"POLYGON ((3.879862753266941 36.92634664178819,...",56Algeria
7,258799889,4736,Algeria,El Menia,town,30.583668,2.883089,4.559333e+04,Subtropic - warm / arid,"POLYGON ((2.873234846555186 30.60876048327147,...",4736Algeria
8,262393185,5427,Algeria,In Guezzam,town,19.566724,5.771700,5.523000e+03,Tropic - warm / arid,"POLYGON ((5.769834985642884 19.56494482203358,...",5427Algeria
9,262963952,3194,Algeria,Benaceur,town,33.110521,6.442111,8.602000e+03,Subtropic - warm / arid,"POLYGON ((6.430187004128339 33.1131937646665, ...",3194Algeria


In [ ]:
test = GHS2000.drop_duplicates('dup', keep=False)
print(len(test))

out = test.drop_duplicates('FID', keep = False)
print(len(out))

In [85]:
# This is the code to use 

test = GHS2000[GHS2000.duplicated(subset=['FID'], keep=False)]
test['dup'] = test['FID'].astype(str) + GHS2000['country']
test.head()

out = test.drop_duplicates('dup', keep=False)
len(out)

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


37

In [4]:
out = test.groupby(['state', 'num']).groups

In [31]:
test.duplicated(['state','num'])

0    False
1    False
2    False
3    False
4     True
5     True
dtype: bool